# Backup via SSH

In [ ]:
import sys
from os import path
import ipywidgets as ipw
from time import strftime
from pprint import pformat
from IPython.display import clear_output

In [ ]:
CONFIG_FN = '.backup.conf'
if path.exists(CONFIG_FN):
    config = eval(open(CONFIG_FN).read())
else:
    config = {'destination':''}

placeholder = "e.g. daint.cscs.ch:mc-backup"
dest_txt = ipw.Text(description="Destination:", placeholder=placeholder, value=config['destination'])
name_txt = ipw.Text(description="Name:", value=strftime("backup_%Y-%m-%d"))

display(dest_txt,name_txt)

In [ ]:
def on_btn_start_clicked(b):
    with output:
        clear_output()
        run_backup(dest_txt.value.strip(), name_txt.value.strip())
btn_start = ipw.Button(description="Start Backup")
btn_start.on_click(on_btn_start_clicked)
display(btn_start)

In [ ]:
def run_backup(dest, name):
    assert len(dest)>0
    assert len(name)>0

    # store config
    config = {'destination': dest}
    open(CONFIG_FN, 'w').write(pformat(config))
    
    run_cmd("mkdir -p /tmp/{name}".format(name=name))
    run_cmd("pg_dump --host=localhost aiidadb | gzip > /tmp/{name}/aiidadb.sql.gz".format(name=name))
    run_cmd("rsync -a /tmp/{name}/ {dest}/{name}/".format(dest=dest, name=name))
    run_cmd("rsync -a --exclude=.postgresql /project/ {dest}/{name}/".format(dest=dest, name=name))
    print("done")

In [ ]:
def run_cmd(cmd):
    print(cmd)
    ! $cmd

In [ ]:
output = ipw.Output()
display(output)